In [ ]:
# ecies_encryption.py
"""
ECIES Encryption/Decryption for SecureMed-LLM

Encrypts and decrypts clinical reports using X25519 and AES-GCM.
"""

import os
from cryptography.hazmat.primitives.asymmetric import x25519
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.ciphers.aead import AESGCM


def encrypt_report_ecies(report_text: str, doctor_public_bytes: bytes) -> dict:
    """Encrypt a clinical report using ECIES (X25519 + AES-GCM)."""
    ephemeral_private_key = x25519.X25519PrivateKey.generate()
    ephemeral_public_key = ephemeral_private_key.public_key()

    doctor_public_key = x25519.X25519PublicKey.from_public_bytes(doctor_public_bytes)
    shared_secret = ephemeral_private_key.exchange(doctor_public_key)

    symmetric_key = HKDF(
        algorithm=hashes.SHA256(),
        length=32,
        salt=None,
        info=b"ecies-securemed"
    ).derive(shared_secret)

    aesgcm = AESGCM(symmetric_key)
    nonce = os.urandom(12)
    ciphertext = aesgcm.encrypt(nonce, report_text.encode(), None)

    return {
        "ephemeral_pub": ephemeral_public_key.public_bytes().hex(),
        "nonce": nonce.hex(),
        "ciphertext": ciphertext.hex()
    }


def decrypt_report_ecies(encrypted_data: dict, doctor_private_bytes: bytes) -> str:
    """Decrypt an ECIES-encrypted clinical report."""
    ephemeral_pub_key = x25519.X25519PublicKey.from_public_bytes(bytes.fromhex(encrypted_data["ephemeral_pub"]))
    doctor_private_key = x25519.X25519PrivateKey.from_private_bytes(doctor_private_bytes)

    shared_secret = doctor_private_key.exchange(ephemeral_pub_key)

    symmetric_key = HKDF(
        algorithm=hashes.SHA256(),
        length=32,
        salt=None,
        info=b"ecies-securemed"
    ).derive(shared_secret)

    aesgcm = AESGCM(symmetric_key)
    nonce = bytes.fromhex(encrypted_data["nonce"])
    ciphertext = bytes.fromhex(encrypted_data["ciphertext"])
    return aesgcm.decrypt(nonce, ciphertext, None).decode()


# Example usage
if __name__ == "__main__":
    # Generate doctor key pair
    doctor_private_key = x25519.X25519PrivateKey.generate()
    doctor_private_bytes = doctor_private_key.private_bytes(
        encoding=None, format=x25519.PrivateFormat.Raw, encryption_algorithm=x25519.NoEncryption()
    )
    doctor_public_bytes = doctor_private_key.public_key().public_bytes(
        encoding=None, format=x25519.PublicFormat.Raw
    )

    # Sample report
    sample_report = "Patient shows mild pneumonia in the left lower lobe."

    # Encrypt
    encrypted = encrypt_report_ecies(sample_report, doctor_public_bytes)
    print("Encrypted:", encrypted)

    # Decrypt
    decrypted = decrypt_report_ecies(encrypted, doctor_private_bytes)
    print("Decrypted:", decrypted)
